In [ ]:
%matplotlib notebook
import pandas as pd
from matplotlib import pyplot as plt

# Data import

In [ ]:
print("hello")

col_names = ["Date","Time","Tag Comment","Motor Vol Flow","Motor Flow In","Motor Flow Out","Motor Surf top","Motor Surf Side","Temp Pt100 RTD winding U", \
            "Temp Pt100 RTD winding V", "Temp Pt100 RTD winding W", "Temp PTC thermistor winding", "Temp Feed-through plate", \
            "Lead intersection", "Temp Cable gland", "Temp Terminal box seal", "Temp Terminal", "Temp Ambient", "Frequency", "Voltage", \
            "Current", "Power factor", "Input power", "Output power", "Torque", "Speed", "Slip"]
print("hello")
df = pd.read_excel('./HDE-INT-MOT-20221028-01_HT8_03_N85z-4_OC2_50Hz_70C_R_000759_221107_100743.GEV.xlsx',header=32)
print("hello")
df.columns = col_names
df = df.drop([0],axis=0)
df.head(5)
print("hello")

# Scatterplot Analysis

In [ ]:
plt.scatter(df["Motor Flow In"],df["Motor Flow Out"])

In [ ]:
plt.scatter(df["Motor Surf top"],df["Motor Surf Side"])

In [ ]:
plt.scatter(df["Temp Pt100 RTD winding U"],df["Temp Ambient"])
plt.plot()


In [ ]:
plt.scatter(df["Time"],df["Temp Pt100 RTD winding U"])